# Algo Trading- Backtrader-Master Template

In this notebook, we will build a backtrader strategy using any trading algorithm


To do:
* Make the tearsheet work
* or use the tearsheet of omegaUI
* Put Teststrategy.py in the algos folder

[1. Step 1 : Loading the algorithm](#1)

[2. Step 2 : Setting the backtester](#2)

[3. Step 3 : Getting the backtest output](#3)

In [2]:
import logging
import os
import pandas as pd
import backtrader as bt
import yfinance as yf
import matplotlib.pyplot as plt
import pyfolio as pf

C:\Users\tatsa\anaconda3\Lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1'></a>
## 1. Step 1 : Loading the algorithm
In this step we load a sample TestStrategy which is created separately in a python file.

In [3]:
from TestStrategy import TestStrategy

<a id='2'></a>
## 2. Step 2 : Setting the backtester
In this step, we set up a backtester, with cash amount, data and strategy name updated.

In [4]:
def backtest():
    #path_dir = os.path.dirname(os.path.realpath(__file__))
    
    cash = 100000
    symbols = ['AAPL','AMZN']
    #start_date = '2018-01-01'
    path_dir = 'Data'
    #strategy = "TestStrategy"
    
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)

    for s in symbols:
            df = pd.read_csv(os.path.join(path_dir, '{}.csv'.format(s)), parse_dates=True, index_col=0)
            data = bt.feeds.PandasData(dataname=df)
            cerebro.adddata(data)
    #data = bt.feeds.PandasData(dataname=yf.download('TSLA', '2018-01-01', '2018-01-10'))
    #cerebro.adddata(data)

    # Strategy
    cerebro.addstrategy(TestStrategy)


    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.SQN, _name='SQN')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    #cerebro.adddata(data)
    # Backtest

    print('Starting Portfolio Value: ', cerebro.broker.getvalue())
    plt.rcParams['figure.figsize']=[10,6]
    plt.rcParams["font.size"]="12"
    # Run over everything
    results = cerebro.run()
    pnl = cerebro.broker.getvalue() - cash
    #cerebro.plot()
    # Print out the final result
    print('Final Portfolio Value: ' , cerebro.broker.getvalue())    
    strat = results[0]
    pyfoliozer = strat.analyzers.getbyname('pyfolio')
    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    return returns, positions, transactions, gross_lev, pnl
    #return results[0], pnl

In [5]:
def test_results():
    returns, positions, transactions, gross_lev, pnl = backtest()
    # returns, pnl = backtest()

In [6]:
test_results()

Starting Portfolio Value:  100000
Final Portfolio Value:  94485.26799999991


<a id='3'></a>
## 3. Step 3 : Getting the backtest output
In this step, we set up a backtester, with cash amount, data and strategy name updated.

In [7]:
returns, positions, transactions, gross_lev, pnl = backtest()

Starting Portfolio Value:  100000
Final Portfolio Value:  94485.26799999991


In [8]:
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    gross_lev=gross_lev,
    live_start_date='2005-05-01',  # This date is sample specific
    round_trips=True)


TypeError: create_full_tear_sheet() got an unexpected keyword argument 'gross_lev'

<a id='4'></a>
## 4. Step 4 : Live Trading
In this step, we set up a backtester, with cash amount, data and strategy name update

In [9]:
import alpaca_backtrader_api
import backtrader as bt
from datetime import datetime

ModuleNotFoundError: No module named 'alpaca_backtrader_api'

In [11]:
#!pip install alpaca_backtrader_api

In [ ]:
ALPACA_API_KEY = PKCDJNRZQFLHN2HB3S8W
ALPACA_SECRET_KEY = CpywEeePTEciIiueHHbBdOnKMlH7P6sKVcKvvbhn
ALPACA_PAPER = True

In [ ]:
class SmaCross(bt.SignalStrategy):
  def __init__(self):
    sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
    crossover = bt.ind.CrossOver(sma1, sma2)
    self.signal_add(bt.SIGNAL_LONG, crossover)

In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

store = alpaca_backtrader_api.AlpacaStore(
    key_id=ALPACA_API_KEY,
    secret_key=ALPACA_SECRET_KEY,
    paper=ALPACA_PAPER
)

if not ALPACA_PAPER:
  broker = store.getbroker()  # or just alpaca_backtrader_api.AlpacaBroker()
  cerebro.setbroker(broker)

DataFactory = store.getdata  # or use alpaca_backtrader_api.AlpacaData
data0 = DataFactory(dataname='AAPL', historical=True, fromdate=datetime(
    2015, 1, 1), timeframe=bt.TimeFrame.Days)
cerebro.adddata(data0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot()